<a href="https://colab.research.google.com/github/Michaelzats/News-finder-and-analiser/blob/main/News_finder_and_analiser_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#News finder and analiser

The following tool is created to help to find the news and topics and to summarise them within few minutes. Besides the model will show the type of the article (if it is postive or negative) so you can have an overview of what is going on

###installing required packages and libraries 

In [1]:
!pip install transformers
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 10.6 MB/s 
     |████████████████████████████████| 86 kB 2.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests
import re


###Setup Summarization Model

In [3]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

###Search for Stock News using Google

In [4]:
monitored_tickers = ["queen","russia"] #Write the keywords with commas (,) to find in the desired articles 

In [5]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=+news+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [6]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'queen': ['/?sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQPAgE',
  '/search?q=news+queen&tbm=nws&ie=UTF-8&gbv=1&sei=6TCZYqjvLcSMtQao7LmQBg',
  '/search?q=news+queen&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUIBygA',
  '/search?q=news+queen&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUICSgC',
  '/search?q=news+queen&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUICigD',
  '/search?q=news+queen&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUICygE',
  'https://maps.google.com/maps?q=news+queen&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUIDCgF',
  '/search?q=news+queen&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQ_AUIDSgG',
  '/advanced_search',
  '/search?q=news+queen&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwjoh-uU4I_4AhVERs0KHSh2DmIQpwUIDw',
  '/s

###Strip out unwanted URLs

In [7]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [8]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [9]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'queen': ['https://www.cbsnews.com/news/queen-elizabeth-ii-platinum-jubilee-celebrations-what-to-expect/',
  'https://www.npr.org/sections/pictureshow/2022/06/02/1102604453/queen-elizabeth-ii-platinum-jubilee-in-photos',
  'https://www.nbcnews.com/news/world/queen-elizabeth-platinum-jubilee-britain-balcony-flyover-parade-rcna31603',
  'https://www.washingtonpost.com/world/interactive/2022/queen-elizabeth-jubilee-reign-photos/',
  'https://www.theguardian.com/news/audio/2022/jun/02/queen-platinum-jubilee-future-of-the-monarchy-podcast',
  'https://www.nbcnews.com/news/world/platinum-pudding-queen-elizabeth-ii-jubilee-dessert-britain-bake-off-rcna31015',
  'https://www.cnbc.com/2022/06/02/buckingham-palace-says-queen-elizabeth-ii-will-miss-platinum-jubilee-church-service-after-feeling-discomfort.html',
  'https://www.bbc.com/news/uk-england-leeds-61661555',
  'https://www.theguardian.com/uk-news/gallery/2022/jun/01/the-queens-other-jubilees-in-pictures',
  'https://www.theartnewspaper.c

###Search and Scrape Cleaned URLs

In [10]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [15]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'queen': ['Watch CBS News \n            By Haley Ott\n           \nUpdated on:  June 2, 2022 / 3:36 PM\n          / CBS News\n         London\xa0— Britain\'s Queen Elizabeth II is celebrating her Platinum Jubilee this year, which is the 70-year anniversary of her reign as queen of the United Kingdom and its 14 Commonwealth nations.\xa0 Already the country\'s longest-reigning monarch — a landmark she reached in 2015 — Elizabeth is the first to celebrate a Platinum Jubilee. Events have been organized around the U.K. and Commonwealth countries to mark the occasion. Here is everything you need to know: Sir Paul McCartney and best-selling author Tina Brown reflect on Queen Elizabeth\'s unprecedented reign in "Her Majesty The Queen: A Gayle King Special." The special airs on the Queen\'s Platinum Jubilee, Thursday, June 2 at 10 p.m. ET on CBS and will stream on the CBS News app and Paramount+ on Friday, June 3. The Platinum Jubilee is the 70th anniversary of a British monarch\'s reign, and 

###Summarise all Articles

In [17]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [18]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'queen': ['Platinum Jubilee will be marked with events across the U.K. and Commonwealth.',
  "Four days of events to celebrate Queen Elizabeth II's 70th year on the throne.",
  'Crowds gather in London to celebrate the Queen’s 70th year on the throne.',
  'Queen of the jet-setting age has made more than 90 state visits. She became Britains longest reigning monarch',
  'It is this defiant ability to remain silent that has served her well, says author.',
  'Jemma Melvin’s lemon Swiss roll and amaretti trifle is official dessert.',
  'Queen chats with great-grandson Prince Louis during Trooping the Color.',
  'Some of those who have met the Queen say she shows a keen interest.',
  'Celebrations to mark the Queen’s 90th birthday have taken place across the country.',
  'Queen Elizabeth II is Britain’s longest-reigning head of state. Here are the news photographs that best define her'],
 'russia': ['Major-General Kanamat Botashev was killed while flying. Russian state news agency says pilo

In [ ]:
#summaries['IPHONE']

###Adding Sentiment Analysis

In [19]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
# sentiment(summaries['MSFT'])

In [20]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'queen': [{'label': 'POSITIVE', 'score': 0.9996929168701172},
  {'label': 'POSITIVE', 'score': 0.9993583559989929},
  {'label': 'POSITIVE', 'score': 0.9996321201324463},
  {'label': 'POSITIVE', 'score': 0.999476969242096},
  {'label': 'POSITIVE', 'score': 0.9997290968894958},
  {'label': 'POSITIVE', 'score': 0.9839185476303101},
  {'label': 'POSITIVE', 'score': 0.9693748950958252},
  {'label': 'POSITIVE', 'score': 0.999681830406189},
  {'label': 'POSITIVE', 'score': 0.9994342923164368},
  {'label': 'POSITIVE', 'score': 0.9990230798721313}],
 'russia': [{'label': 'NEGATIVE', 'score': 0.9912800192832947},
  {'label': 'POSITIVE', 'score': 0.995354175567627},
  {'label': 'NEGATIVE', 'score': 0.9909830093383789},
  {'label': 'POSITIVE', 'score': 0.9668781757354736},
  {'label': 'NEGATIVE', 'score': 0.9766238331794739},
  {'label': 'POSITIVE', 'score': 0.9668781757354736},
  {'label': 'POSITIVE', 'score': 0.8454402685165405},
  {'label': 'NEGATIVE', 'score': 0.9935927987098694},
  {'label':

In [21]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [23]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['queen',
  'Platinum Jubilee will be marked with events across the U.K. and Commonwealth.',
  'POSITIVE',
  0.9996929168701172,
  'https://www.cbsnews.com/news/queen-elizabeth-ii-platinum-jubilee-celebrations-what-to-expect/'],
 ['queen',
  "Four days of events to celebrate Queen Elizabeth II's 70th year on the throne.",
  'POSITIVE',
  0.9993583559989929,
  'https://www.npr.org/sections/pictureshow/2022/06/02/1102604453/queen-elizabeth-ii-platinum-jubilee-in-photos'],
 ['queen',
  'Crowds gather in London to celebrate the Queen’s 70th year on the throne.',
  'POSITIVE',
  0.9996321201324463,
  'https://www.nbcnews.com/news/world/queen-elizabeth-platinum-jubilee-britain-balcony-flyover-parade-rcna31603'],
 ['queen',
  'Queen of the jet-setting age has made more than 90 state visits. She became Britains longest reigning monarch',
  'POSITIVE',
  0.999476969242096,
  'https://www.washingtonpost.com/world/interactive/2022/queen-elizabeth-jubilee-reign-photos/'],
 ['queen',
  'It is this

In [24]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [25]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['queen',
  'Platinum Jubilee will be marked with events across the U.K. and Commonwealth.',
  'POSITIVE',
  0.9996929168701172,
  'https://www.cbsnews.com/news/queen-elizabeth-ii-platinum-jubilee-celebrations-what-to-expect/'],
 ['queen',
  "Four days of events to celebrate Queen Elizabeth II's 70th year on the throne.",
  'POSITIVE',
  0.9993583559989929,
  'https://www.npr.org/sections/pictureshow/2022/06/02/1102604453/queen-elizabeth-ii-platinum-jubilee-in-photos'],
 ['queen',
  'Crowds gather in London to celebrate the Queen’s 70th year on the throne.',
  'POSITIVE',
  0.9996321201324463,
  'https://www.nbcnews.com/news/world/queen-elizabeth-platinum-jubilee-britain-balcony-flyover-parade-rcna31603'],
 ['queen',
  'Queen of the jet-setting age has made more than 90 state visits. She became Britains longest reigning monarch',
  'POSITIVE',
  0.999476969242096,
  'https://www.washingtonpost.com/world/interactive/2022/queen-eliza

In [ ]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)